In [1]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
#tf.enable_eager_execution()
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
import pickle

In [2]:
with open('encodedStates.pickle', 'rb') as handle:
    data = pickle.load(handle)

In [5]:
def df_to_np(dataframe,shuffle=True,batch_size=32,target="HHVEHCNT"):
    dataframe = dataframe.copy()
    labels = dataframe.pop(target)
    return dataframe.values, labels.values

In [31]:
class TransferLearn:
    
    def __init__(self, fromStateData, toStateData):
        self.data = data
        self.fromState = fromStateData
        self.toState = toStateData
        
        self.train_X_from, self.train_y_from = df_to_np(fromStateData)
        self.train_X_to, self.train_y_to = df_to_np(toStateData)
    
        
    def getFromModelWeights(self, batch_size=100, validation_split=0.2, epochs=5):
        model = tf.keras.Sequential([
          tf.keras.layers.Dense(128,activation="relu"),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(64,activation="relu"),
          tf.keras.layers.Dense(4,activation="softmax")
        ])

        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        model.fit(x=self.train_X_from, y = self.train_y_from,epochs=5, batch_size = batch_size, validation_split=validation_split, shuffle=True)
        
        return model.layers[0].get_weights()
    
    def transfer(self, weights=None, trainable=True, batch_size=100, validation_split=0.2, epochs=5):
        model = tf.keras.Sequential([
          tf.keras.layers.Dense(128,activation="relu", weights=weights, trainable=trainable),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(128),
          tf.keras.layers.Dense(64,activation="relu"),
          tf.keras.layers.Dense(4,activation="softmax")
        ])
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

        model.fit(x=self.train_X_to, y = self.train_y_to, epochs=epochs, batch_size = batch_size, validation_split=validation_split, shuffle=True)
        
        return model
    
    def compare(self):
        pass

In [32]:
MA_to_CA = TransferLearn(data["MA"],data["CA"])

In [33]:
weightsMA = MA_to_CA.getFromModelWeights()
weightsMA

Train on 2568 samples, validate on 643 samples
Epoch 1/5
2568/2568 [==============================] - 1s 248us/sample - loss: 1.0145 - accuracy: 0.5810 - val_loss: 1.0840 - val_accuracy: 0.5568
Epoch 2/5
2568/2568 [==============================] - 0s 40us/sample - loss: 0.8518 - accuracy: 0.6651 - val_loss: 1.0318 - val_accuracy: 0.5677
Epoch 3/5
2568/2568 [==============================] - 0s 41us/sample - loss: 0.7980 - accuracy: 0.6939 - val_loss: 1.0246 - val_accuracy: 0.5956
Epoch 4/5
2568/2568 [==============================] - 0s 40us/sample - loss: 0.7587 - accuracy: 0.6939 - val_loss: 1.0294 - val_accuracy: 0.5956
Epoch 5/5
2568/2568 [==============================] - 0s 39us/sample - loss: 0.7432 - accuracy: 0.7076 - val_loss: 1.0719 - val_accuracy: 0.5661


[array([[ 0.06741601,  0.0986798 , -0.10876605, ..., -0.17093259,
         -0.20180637, -0.16391198],
        [-0.11321911, -0.03299568, -0.00843035, ...,  0.01787489,
         -0.08031259,  0.09815881],
        [-0.08905667, -0.14832033,  0.13841173, ..., -0.09980188,
          0.04465599,  0.1549582 ],
        ...,
        [ 0.0529953 , -0.13900506, -0.15134601, ...,  0.01721604,
         -0.0318675 ,  0.22476268],
        [ 0.04599365, -0.06034075,  0.00293593, ..., -0.1239102 ,
          0.05726321, -0.10652932],
        [-0.01877234, -0.07873057,  0.04842931, ..., -0.19494025,
          0.16751519, -0.11740341]], dtype=float32),
 array([ 0.00969146,  0.00489551,  0.01337628,  0.00279886,  0.01624493,
         0.01873527,  0.02939535, -0.00895472, -0.00413715,  0.0173722 ,
         0.        ,  0.01324966,  0.00459244, -0.01783877,  0.00391392,
         0.00641873, -0.00608421,  0.01460391,  0.00285259,  0.01955558,
         0.00552241,  0.        ,  0.05433843, -0.01413904,  0.002

In [34]:
modelCA_frozenTransfer = MA_to_CA.transfer(weights=weightsMA,trainable=False)

Train on 124793 samples, validate on 31199 samples
Epoch 1/5
124793/124793 [==============================] - 5s 42us/sample - loss: 0.9287 - accuracy: 0.5761 - val_loss: 0.9321 - val_accuracy: 0.5878
Epoch 2/5
124793/124793 [==============================] - 5s 37us/sample - loss: 0.9181 - accuracy: 0.5829 - val_loss: 0.9401 - val_accuracy: 0.5790
Epoch 3/5
124793/124793 [==============================] - 5s 37us/sample - loss: 0.9152 - accuracy: 0.5853 - val_loss: 0.9277 - val_accuracy: 0.5825
Epoch 4/5
124793/124793 [==============================] - 4s 36us/sample - loss: 0.9129 - accuracy: 0.5876 - val_loss: 0.9270 - val_accuracy: 0.5871
Epoch 5/5
124793/124793 [==============================] - 4s 32us/sample - loss: 0.9122 - accuracy: 0.5875 - val_loss: 0.9294 - val_accuracy: 0.5803


In [35]:
modelCA_frozenTransfer.layers[0].get_weights()

[array([[ 0.06741601,  0.0986798 , -0.10876605, ..., -0.17093259,
         -0.20180637, -0.16391198],
        [-0.11321911, -0.03299568, -0.00843035, ...,  0.01787489,
         -0.08031259,  0.09815881],
        [-0.08905667, -0.14832033,  0.13841173, ..., -0.09980188,
          0.04465599,  0.1549582 ],
        ...,
        [ 0.0529953 , -0.13900506, -0.15134601, ...,  0.01721604,
         -0.0318675 ,  0.22476268],
        [ 0.04599365, -0.06034075,  0.00293593, ..., -0.1239102 ,
          0.05726321, -0.10652932],
        [-0.01877234, -0.07873057,  0.04842931, ..., -0.19494025,
          0.16751519, -0.11740341]], dtype=float32),
 array([ 0.00969146,  0.00489551,  0.01337628,  0.00279886,  0.01624493,
         0.01873527,  0.02939535, -0.00895472, -0.00413715,  0.0173722 ,
         0.        ,  0.01324966,  0.00459244, -0.01783877,  0.00391392,
         0.00641873, -0.00608421,  0.01460391,  0.00285259,  0.01955558,
         0.00552241,  0.        ,  0.05433843, -0.01413904,  0.002